In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install langchain_core
!pip install langchain_groq
!pip install langchain
!pip install langchain-huggingface

In [4]:
import os

from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

In [5]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [6]:
llm = init_chat_model(model="llama3-8b-8192", model_provider="groq")

In [10]:
prompt = PromptTemplate(
    template="""
Detect the language of each text segment from the given input and wrap it in the format <ABCD>text</ABCD>, 
where ABCD is the appropriate ISO 639-3 language code. Follow these rules:
Use the ISO 639-3 language code if available.
If a language is uncoded, use <mis>.
If multiple languages are mixed in one segment, use <mul>.
If the language is undetermined, use <und>.
If the text has no linguistic content, such as symbols or sounds, use <zxx>.
Here is the user text: {text} \n
""",
    input_variables=["text"],
)
chain = prompt | llm

In [11]:
result = chain.invoke({"text": """
=====================================
           EU BANKING AGREEMENT
=====================================

Dokumenttyp: Multilinguale Banktransaktionsvereinbarung  
Tipo de documento: Acuerdo bancario multilingüe  
Type de document : Accord bancaire multilingue  
Тип документа: Многоязычное банковское соглашение  

Kunde / Cliente / Client / Клиент: 
-------------------------------------
Name: Johann Müller (Germany)  
Nom: Jean Dupont (France)  
Nombre: Carlos Fernández (Spain)  
Имя: Иван Петров (Russia)  

Kontodaten / Datos de cuenta / Données du compte / Данные счета:
-------------------------------------
Bankname / Nombre del banco / Nom de la banque / Название банка: 
Deutsche Bank AG / Banco Santander / BNP Paribas / Сбербанк

SWIFT-Code: DEUTDEFFXXX / BSCHESMMXXX / BNPAGFRPPXXX / SABRRUMMXXX

IBAN: DE89 3704 0044 0532 0130 00 / ES91 2100 0418 4502 0005 1332 / FR76 3000 6000 0112 3456 7890 189 / RU21 044525225 12345678901234

Transaktionsdetails / Detalles de la transacción / Détails de la transaction / Детали транзакции:
-------------------------------------
Betrag / Cantidad / Montant / Сумма: €10,000,000.00  
Währung / Moneda / Devise / Валюта: EUR  
Transaktionsreferenz / Referencia de transacción / Référence de transaction / Номер транзакции: TXN-EU-2025-987654

Spezielle Bedingungen / Condiciones especiales / Conditions spéciales / Специальные условия:
-------------------------------------
1. Diese Transaktion unterliegt den Bestimmungen der Europäischen Zentralbank (EZB).  
   Esta transacción está sujeta a la normativa del Banco Central Europeo (BCE).  
   Cette transaction est soumise à la réglementation de la Banque centrale européenne (BCE).  
   Эта транзакция подчиняется нормативам Европейского центрального банка (ЕЦБ).  

2. Multinationale Überprüfung durch SWIFT und regulatorische Behörden erforderlich.  
   Revisión multinacional requerida por SWIFT y autoridades regulatorias.  
   Vérification multinationale requise par SWIFT et les autorités réglementaires.  
   Многонациональная проверка требуется SWIFT и регулирующими органами.  

Genehmigung / Aprobación / Approbation / Одобрение:
-------------------------------------
Unterschrift / Firma / Signature / Подпись: 
Johann Müller (Deutsche Bank) / Carlos Fernández (Banco Santander) / Jean Dupont (BNP Paribas) / Иван Петров (Сбербанк)

Datum / Fecha / Date / Дата: 05 März 2025 / 05 marzo 2025 / 05 mars 2025 / 05 марта 2025

=====================================
Hinweis / Aviso / Remarque / Примечание: Dieses Dokument unterliegt internationalen Bankvorschriften.
====================================="""})

In [12]:
result.content

'Here is the output with the detected language wrapped in the format `<ABCD>text</ABCD>`:\n\n```\n<de>EU BANKING AGREEMENT</de>\n<Dokumenttyp: Multilinguale Banktransaktionsvereinbarung </mul><Tipo de documento: Acuerdo bancario multilingüe </mul><Type de document : Accord bancaire multilingue </mul><Тип документа: Многоязычное банковское соглашение </mul>\n\n<Kunde / Cliente / Client / Клиент: \n-------------------------------------\n<de>Name: Johann Müller (Germany) </de><fr>Nom: Jean Dupont (France) </fr><es>Nombre: Carlos Fernández (Spain) </es><ru>Имя: Иван Петров (Russia) </ru>\n\n<Kontodaten / Datos de cuenta / Données du compte / Данные счета:\n-------------------------------------\n<de>Bankname / Nombre del banco / Nom de la banque / Название банка: \n<de>Deutsche Bank AG / </de><es>Banco Santander / </es><fr>BNP Paribas / </fr><ru>Сбербанк / </ru>\n\n<de>SWIFT-Code: DEUTDEFFXXX / </de><es>BSCHESMMXXX / </es><fr>BNPAGFRPPXXX / </fr><ru>SABRRUMMXXX / </ru>\n\n<de>IBAN: DE89 370